# Hello, quantum world! en ProjectQ

En este notebook mostraremos cómo ProjectQ puede ser usado para simular circuitos cuánticos.

## Simulando un circuito

ProjectQ permite simular circuitos cuánticos en el ordenador local. Veamos cómo conseguirlo con un ejemplo sencillo. Este primer circuito simplemente aplica una puerta de Hadamard a un qubit y lo mide. El resultado será, aleatoriamente, un 0 un 1. Ejecutando el código en distintas ocasiones, se obtendrá 0 o 1 con un 50% de probabilidad.

Nótese que hemos definido una función que es la que encapsula todas las puertas del circuito. Esta función recibe un *engine* (un motor de ejecución, que puede ser un simulador, un ordenador cuántico real...) y devuelve el estado de los qubits tras la aplicación de las puertas. Esto es útil, porque nos permite reutilizar el mismo circuito en distintos entornos de ejecución.

In [ ]:
import projectq
from projectq.ops import Measure, H

def hello_world(eng):
    
    qubit = eng.allocate_qubit() # Declaramos un qubit
    H | qubit                    # Aplicamos la puerta H al qubit
    Measure | qubit              # Medimos el qubit
    eng.flush()                  # Mandamos todas las instrucciones al engine para que las ejecute
    
    return qubit                 # Devolvemos el estado del qubit


eng = projectq.MainEngine()  # Creamos un 'engine' que nos permitirá simular nuestros circuitos
qubit = hello_world(eng)     # Usamos el simulador para ejecutar el circuito

print(int(qubit))            # Convertimos el valor medido en el qubit a entero y lo mostramos 

Vamos a ejecutar 1000 veces el circuito anterior, calculando el número de aparaciones de cada resultado.

In [ ]:
resultados = {0:0,1:0}

for _ in range(1000):
    qubit = hello_world(eng)       
    valor = int(qubit)
    resultados[valor]+= 1

print(resultados)
    

Al tratarse de un simulador, podemos acceder a la función de onda de los qubits, algo que en los ordenadores cuánticos reales no es posible. Veamos cómo se hace usando el mismo circuito de antes. Es importante darse cuenta de que **no realizamos la medida** antes de acceder a las amplitudes y probabilidades, porque eso haría colapsar la función de onda en un resultado concreto. Sin embargo, sí que debemos medir después, para evitar un **error** por parte del compilador (todos los qubits de todos los circuitos deben medirse o devolverse al estado $|0\rangle$ antes de terminar).

In [ ]:
qubit = eng.allocate_qubit()
H | qubit
eng.flush()

amp = {}
prob = {}
for val in ['0','1']:
    amp[val]  = eng.backend.get_amplitude(val,qubit)
    prob[val] = eng.backend.get_probability(val,qubit)
    
print('Amplitudes:', amp)
print('Probabilidades', prob)

Measure | qubit